<a href="https://colab.research.google.com/github/madanmohan-venkatesan/notebooks/blob/main/2011_03567.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [13]:
rng = np.random.default_rng()
null_vectors=rng.multinomial(1,[1/100]*100)

In [24]:
theta=rng.dirichlet([1/100]*100)
alt_vectors=rng.multinomial(1,theta)

ValueError: pvals < 0, pvals > 1 or pvals contains NaNs

In [23]:
alt_vectors

array([1.75837729e-027, 1.83668449e-014, 3.13165134e+001, 3.82908028e-007,
       7.10016801e-016, 8.29221050e-047, 1.42600291e-073, 5.82620035e-024,
       1.16089506e-004, 1.67306305e-028, 3.60866016e-105, 5.55329701e-015,
       5.89862626e-038, 2.60995821e-120, 1.21434067e-093, 6.66117335e-093,
       4.41417165e-011, 6.70808287e-008, 3.25978966e-020, 1.22951557e-037,
       7.05109799e-009, 2.27882154e-016, 6.12940678e-035, 2.70387648e-045,
       3.77605617e-015, 4.16461737e-007, 1.42780819e-010, 6.83914946e-003,
       5.96691492e-055, 4.32984806e-062, 2.21978928e-005, 8.04622720e-059,
       3.09006599e-010, 4.38944886e-031, 1.23868827e-040, 5.16542586e-009,
       1.89613227e-028, 3.99027744e-029, 6.77645922e-119, 1.13561436e-048,
       3.47432418e-026, 1.25616290e-091, 2.40161415e-004, 4.75953155e-024,
       4.06731063e-070, 4.89940636e-024, 2.01500006e-097, 2.29500676e-105,
       4.79705899e-176, 1.60885588e-087, 2.78698455e-083, 1.60686567e-013,
       1.99589519e-014, 2